In [1]:
import networkx as nx
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from flinspect.gen_dependency_graph import gen_module_dependency_graph, gen_call_graph
from flinspect.parse_tree import ParseTree

# Generate Call Graph

# Plot Module Dependency Graph

In [4]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/",
    #"/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6"
]

parse_tree_files = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [5]:
G = gen_module_dependency_graph(parse_tree_files)
assert nx.is_directed_acyclic_graph(G)

In [6]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [7]:
def get_color(G, node):
    source_name = G.nodes[node].get('source_name','')
    source_path = path_names.get(source_name.lower(),'')
    
    node_color, edge_color = "grey", "lightgrey"
    if "/config_src/infra" in source_path:
        node_color, edge_color = "darkred", "red"    
    elif "/MARBL/" in source_path:
        node_color, edge_color = "purple", "orchid"
    elif "/CVMix-src/" in source_path:
        node_color, edge_color = "chocolate", "sandybrown"
    elif "/GSW-Fortran/" in source_path:
        node_color, edge_color = "royalblue", "cornflowerblue"
    elif "/MOM6/" in source_path:
        node_color, edge_color = "lightseagreen", "turquoise" # for core MOM6        
    elif "/FMS/" in source_path:
        node_color, edge_color = "orange", "gold"
        
    return node_color, edge_color

In [8]:
# Determine Node positions using 'fdp' layout
pos = nx.nx_agraph.graphviz_layout(G, prog='fdp')

In [9]:
from pyvis.network import Network

In [10]:
net = Network(notebook=True, directed=True, height='800px', cdn_resources='in_line')

net.set_options("""
var options = {
  "physics": { "enabled": false },
  "nodes": { "shape": "dot", "size": 10 },
  "edges": { "arrows": { "to": { "enabled": true, "scaleFactor": 0.3 }    } }
}
""")

# Add nodes with positions
for node in G.nodes():
    node_color, _ = get_color(G, node)
    x, y = pos[node]
    net.add_node(str(node), color=node_color, label=str(node), x=x, y=y, fixed=True)  # y flipped for browser coord system

# Add edges
for source, target in G.edges():
    _, edge_color = get_color(G, source)
    net.add_edge(str(source), str(target), color=edge_color)


In [11]:
# 3232 or 3164
len(G.edges)

327

In [12]:
# net.show("module_dependency.html")

In [13]:
from flinspect.parse_tree import *
from flinspect.utils import *

# Plot Call Graph

In [14]:
nr = gen_call_graph(parse_tree_files);

Could not find callee mpp_error in any used module of grid_mod for call in get_grid_version in /glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/grid.o_ptree, line: | | | | | | | | | | | ProcedureDesignator -> Name = 'mpp_error'
Could not find callee read_data in any used module of grid_mod for call in get_grid_ntiles in /glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/grid.o_ptree, line: | | | | | | | | | ProcedureDesignator -> Name = 'read_data'
Could not find callee read_data in any used module of grid_mod for call in get_grid_size_for_all_tiles in /glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/grid.o_ptree, line: | | | | | | | | | ProcedureDesignator -> Name = 'read_data'
Could not find callee mpp_error in any used module of grid_mod for call in get_grid_size_for_one_tile in /glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/grid.o_ptree, line: | | | | | | | | | ProcedureDesignator -> Name = 'mpp_error'
Could not find callee read_data in any used module of grid_mod f

In [15]:
from flinspect.parse_node import Module, Program, Subprogram, Subroutine, Function

In [16]:
len(path_names)

207

In [17]:
len(nr._store[Module])

73

In [18]:
len(nr._store[Subroutine])

1693

In [19]:
len(nr._store[Function])

649

In [20]:
len(nr._store[Subprogram])

KeyError: <class 'flinspect.parse_node.Subprogram'>

In [21]:
from flinspect.parse_node import Callable

In [22]:
import gc
num_edges = 0
for obj in gc.get_objects():
    if isinstance(obj, Callable):
        if obj.callees:
            callee_names = [callee.name for callee in obj.callees]
            if num_edges < (cutoff := 20):
                print(f"caller: {obj}, callee: {callee_names}")
            elif num_edges == cutoff:
                print("...")
        
            num_edges += len(callee_names)

print(f"\nTotal edges (i.e., calls) in call graph: {num_edges}")

caller: register_variable_attribute_0d, callee: ['check_netcdf_code', 'set_netcdf_mode']
caller: register_variable_attribute_1d, callee: ['check_netcdf_code', 'set_netcdf_mode']
caller: get_global_attribute_0d, callee: ['check_netcdf_code']
caller: get_global_attribute_1d, callee: ['check_netcdf_code']
caller: get_variable_attribute_0d, callee: ['check_netcdf_code']
caller: get_variable_attribute_1d, callee: ['check_netcdf_code']
caller: compressed_write_3d, callee: ['compressed_write_4d']
caller: netcdf_file_close_wrap, callee: ['netcdf_file_close']
caller: netcdf_add_variable_wrap, callee: ['netcdf_add_variable']
caller: netcdf_save_restart_wrap, callee: ['netcdf_save_restart']
caller: flush_file, callee: ['check_netcdf_code']
caller: get_peset, callee: ['expand_peset']
caller: mpp_sync, callee: ['increment_current_clock', 'system_clock_mpi']
caller: mpp_sync_self, callee: ['increment_current_clock', 'system_clock_mpi']
caller: mpp_error_ia, callee: ['mpp_error_basic']
caller: mpp_er